In [ ]:
# working with JSON fields

In [ ]:
# import libs (or think of this as downloading python python apps)
import psycopg2
from jettings import Jettings
import os

In [ ]:
# Access database credentials

password_file="~/database.secrets"

password_file=os.path.expanduser(password_file)
password_file=os.path.abspath(password_file)

if(not os.path.exists(password_file)):
    print("File doesn't exist!")
    my_settings=Jettings(password_file)
    my_settings.sets(['tutorial','db_host'],"*****")
    my_settings.sets(['tutorial','db_username'],"*****")
    my_settings.sets(['tutorial','db_password'],"*****")
    my_settings.sets(['tutorial','db_name'],"*****")
    my_settings.sets(['tutorial','db_schema'],"*****")
    my_settings.sets(['tutorial','editlock'],"*****")
else:
    my_settings=Jettings(password_file)
    print("File exists!")
    edit_lock=my_settings.exists(['tutorial','editlock'])
    if(edit_lock):
        message=("Please add connection details to file: "+str(password_file)+"\nand delete the edit lock.")
        print(message)
    else:
        db_host=my_settings.gets(['tutorial','db_host'])
        db_username=my_settings.gets(['tutorial','db_username'])
        db_password=my_settings.gets(['tutorial','db_password'])
        db_name=my_settings.gets(['tutorial','db_name'])
        db_schema=my_settings.gets(['tutorial','db_schema'])
        print("Connections credentials retrieved!")


In [ ]:
# connection variables
conn = None

# catch external errors
try:
    conn = psycopg2.connect(host=db_host, 
                           user=db_username,
                           password=db_password,
                           database=db_name)
    print("Connecting to the database...")
    conn.autocommit = True
except (Exception, psycopg2.DatabaseError) as banana:
    print(banana)

In [ ]:
# Target the right table, schema

existing_schema = "zokitools"
new_table = "orders"

In [ ]:
# Create a new table, with JSON field
# Create SQL statment with f-strings

create_table = f'''
CREATE TABLE IF NOT EXISTS {existing_schema}.{new_table}(
id SERIAL NOT NULL PRIMARY KEY,
info json NOT NULL
)
'''
print(create_table)

In [ ]:
# ask the connection for a cursor, then ask the cursor to run your sql statement

cur=conn.cursor()
cur.execute(create_table)

In [ ]:
########### INSERTING JSON DATA ###########
###########################################

In [ ]:
# Create a query with variables

field_tuple_string ='(info)'
json_string='{"customer":"John Doe","items":{"product":"Beer","qty":6}}'

# Create insert query using f-strings and the variables above...

insert_sql = f'''
INSERT INTO {existing_schema}.{new_table} {field_tuple_string} VALUES ('{json_string}')
'''
print(insert_sql)

In [ ]:
# run the SQL statement

cur.execute(insert_sql)

In [ ]:
# Construct strings with escaped { }

insert_sql2=f'''
INSERT INTO {existing_schema}.{new_table} (info) VALUES ('{{"customer":"John Doe","items":{{"product":"Beer","qty":6}}}}')
'''
print(insert_sql2)

In [ ]:
# run the SQL statement

cur.execute(insert_sql2)

In [ ]:
insert_sql3 = f'''
INSERT INTO {existing_schema}.{new_table} (info) VALUES 
      ('{{ "customer": "Lily Bush", "items": {{"product": "Diaper","qty": 24}}}}'),
      ('{{ "customer": "Josh William", "items": {{"product": "Toy Car","qty": 1}}}}'),
      ('{{ "customer": "Mary Clark", "items": {{"product": "Toy Train","qty": 2}}}}');
'''
print(insert_sql3)

In [ ]:
cur.execute(insert_sql3)

In [ ]:
# Include a new field "Country" for some new records

insert_sql4 = f'''
INSERT INTO {existing_schema}.{new_table} (info) VALUES 
      ('{{ "customer": "Lily Bush", "country": "US", "items": {{"product": "Diaper","qty": 24}}}}'),
      ('{{ "customer": "Josh William", "country": "NL", "items": {{"product": "Toy Car","qty": 1}}}}'),
      ('{{ "customer": "Mary Clark", "country": "US", "items": {{"product": "Toy Train","qty": 2}}}}');
'''
print(insert_sql4)

In [ ]:
cur.execute(insert_sql4)

In [ ]:
# CREATE JSON READ QUERY

select_json = f' select * from {existing_schema}.{new_table} where info->>\'country\'=\'US\';'
print(select_json)

In [ ]:
cur.execute(select_json)
customers=cur.fetchall()

In [ ]:
for a_customer in customers:
    print(a_customer)